# Using MobileNetV2 Model

In [1]:
#import all necessary modules
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import matplotlib.pyplot as plt

In [10]:
# MobileNetV2 is commonly trained on 224x224 images
height = 224
width = 224

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(height, width, 3))

In [11]:
# --- Data Loading and Augmentation ---
train_dir = "dataset/training"
validation_dir = "dataset/validation"
batch_size = 8

In [12]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # Use MobileNetV2's preprocess_input
    rotation_range=90,
    horizontal_flip=True,
    vertical_flip=True
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    batch_size=batch_size
)


Found 2774 images belonging to 2 classes.


In [13]:
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 592 images belonging to 2 classes.


In [14]:
def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    # Freeze the layers of the base model
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom top layers for classification
    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        x = Dense(fc, activation='relu')(x)
        x = Dropout(dropout)(x)

    predictions = Dense(num_classes, activation='softmax')(x)

    finetune_model = Model(inputs=base_model.input, outputs=predictions)
    return finetune_model

In [15]:
class_list = ['Real', 'Fake']
FC_Layers = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(
    base_model,
    dropout=dropout,
    fc_layers=FC_Layers,
    num_classes=len(class_list)
)

In [16]:
num_epochs = 20
num_train_images = 2774 # From your notebook's output

# Callbacks for saving the best model and stopping early
checkpoint = ModelCheckpoint(
    "MobileNetV2_model.h5",
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)
early = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=40,
    verbose=1,
    mode="auto"
)

# Compile the model
optimizer = optimizers.SGD(learning_rate=0.001, momentum=0.9)
finetune_model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizer,
    metrics=['accuracy']
)

In [17]:
finetune_model.fit(
    train_generator,
    steps_per_epoch=num_train_images // batch_size,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=1, # As in the original notebook
    callbacks=[checkpoint, early]
)

c:\Users\PRATYAOY SARKAR\FinalYrProject\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 637ms/step - accuracy: 0.5816 - loss: 8.8861
Epoch 1: val_accuracy improved from None to 0.75000, saving model to MobileNetV2_model.h5


346/346 ━━━━━━━━━━━━━━━━━━━━ 232s 652ms/step - accuracy: 0.6233 - loss: 4.7603 - val_accuracy: 0.7500 - val_loss: 0.6321
Epoch 2/20
  1/346 ━━━━━━━━━━━━━━━━━━━━ 3:17 573ms/step - accuracy: 0.8750 - loss: 0.4806

c:\Users\PRATYAOY SARKAR\FinalYrProject\myenv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy did not improve from 0.75000
346/346 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8750 - loss: 0.4806 - val_accuracy: 0.7500 - val_loss: 0.6269
Epoch 3/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 556ms/step - accuracy: 0.6412 - loss: 0.6369
Epoch 3: val_accuracy improved from 0.75000 to 0.87500, saving model to MobileNetV2_model.h5


346/346 ━━━━━━━━━━━━━━━━━━━━ 195s 563ms/step - accuracy: 0.6612 - loss: 0.5998 - val_accuracy: 0.8750 - val_loss: 0.6134
Epoch 4/20
  1/346 ━━━━━━━━━━━━━━━━━━━━ 3:24 592ms/step - accuracy: 0.7500 - loss: 0.5239
Epoch 4: val_accuracy did not improve from 0.87500
346/346 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7500 - loss: 0.5239 - val_accuracy: 0.8750 - val_loss: 0.6099
Epoch 5/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 558ms/step - accuracy: 0.7147 - loss: 0.5253
Epoch 5: val_accuracy improved from 0.87500 to 1.00000, saving model to MobileNetV2_model.h5


346/346 ━━━━━━━━━━━━━━━━━━━━ 196s 566ms/step - accuracy: 0.7202 - loss: 0.5255 - val_accuracy: 1.0000 - val_loss: 0.3769
Epoch 6/20
  1/346 ━━━━━━━━━━━━━━━━━━━━ 3:14 563ms/step - accuracy: 0.5000 - loss: 0.5482
Epoch 6: val_accuracy did not improve from 1.00000
346/346 ━━━━━━━━━━━━━━━━━━━━ 1s 983us/step - accuracy: 0.5000 - loss: 0.5482 - val_accuracy: 1.0000 - val_loss: 0.3725
Epoch 7/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 565ms/step - accuracy: 0.7094 - loss: 0.5245
Epoch 7: val_accuracy did not improve from 1.00000
346/346 ━━━━━━━━━━━━━━━━━━━━ 196s 566ms/step - accuracy: 0.7180 - loss: 0.5092 - val_accuracy: 0.6250 - val_loss: 0.6054
Epoch 8/20
  1/346 ━━━━━━━━━━━━━━━━━━━━ 3:18 574ms/step - accuracy: 0.7500 - loss: 0.3836
Epoch 8: val_accuracy did not improve from 1.00000
346/346 ━━━━━━━━━━━━━━━━━━━━ 1s 995us/step - accuracy: 0.7500 - loss: 0.3836 - val_accuracy: 0.6250 - val_loss: 0.5956
Epoch 9/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 556ms/step - accuracy: 0.7799 - loss: 0.4340
Epoch 9: va

In [18]:
MODEL_PATH = "MobileNetV2_model.h5" 

model = load_model(MODEL_PATH)
true_labels = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

predictions = model.predict(validation_generator)
predicted_labels = np.argmax(predictions, axis=1)


74/74 ━━━━━━━━━━━━━━━━━━━━ 24s 304ms/step


In [20]:
from sklearn.metrics import classification_report, accuracy_score

In [21]:
print("--- Model Evaluation Report ---")
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy:.4f}\n")

# Print precision, recall, f1-score for each class
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=class_labels))

--- Model Evaluation Report ---
Accuracy: 0.6402

Classification Report:
              precision    recall  f1-score   support

        fake       0.60      0.84      0.70       296
        real       0.74      0.44      0.55       296

    accuracy                           0.64       592
   macro avg       0.67      0.64      0.62       592
weighted avg       0.67      0.64      0.62       592

